Descargamos los archivos de la base de datos publica de YELP de su pagina


oficial https://www.yelp.com/dataset

In [ ]:
!pip install textblob

In [ ]:
#librerias
import pandas as pd
import json
from textblob import TextBlob
import tarfile
import os
import sys
import re
import random
import string
import csv

Después de descargar el archivo desde la pagina oficial de Yelp  procedemos a descomprimir los archivos que lo conforman

In [ ]:
# Cargamos la ruta del archivo .tgz
ruta_tgz = '/content/drive/MyDrive/data/datos en bruto/yelp_dataset.tgz'

# Cargamos la ruta de extracción donde se guardara el archivo
ruta_extraccion = '/content/sample_data'

# Extraemos los archivos que comforman el archivo .tgz
try:
    with tarfile.open(ruta_tgz, 'r:gz') as archivo_tgz:
        archivo_tgz.extractall(path=ruta_extraccion)
    resultado = f"Archivos extraídos exitosamente en: {ruta_extraccion}"
except Exception as e:
    resultado = f"Error al extraer los archivos: {e}"

resultado


'Archivos extraídos exitosamente en: /content/sample_data'


Cambiamos el formato de los archivos YELP de .json a csv

In [ ]:
# Cargamos las rutas de los archivos .json
archivos = {
    "business": "/content/sample_data/yelp_academic_dataset_business.json",
    "checkin": "/content/sample_data/yelp_academic_dataset_checkin.json",
    "review": "/content/sample_data/yelp_academic_dataset_review.json",
    "tip": "/content/sample_data/yelp_academic_dataset_tip.json",
    "user": "/content/sample_data/yelp_academic_dataset_user.json"
}

# Establesemos el tamaño de chunk para procesar los archivos en partes y evitar la saturacion de la memoria
chunk_size = 10000

# Convertimos cada archivo de .json a .csv usando chunks
for nombre, ruta in archivos.items():
    try:
        # Definimos donde guardaremos el archivo .csv
        ruta_salida = f"/content/sample_data/{nombre}.csv"

        # Usamos chunks para procesar el archivo
        with pd.read_json(ruta, lines=True, chunksize=chunk_size) as reader:
            for i, chunk in enumerate(reader):
                mode = 'w' if i == 0 else 'a'
                header = i == 0
                chunk.to_csv(ruta_salida, index=False, mode=mode, header=header)

        print(f"Archivo {nombre} convertido y guardado como {ruta_salida}.")
    except Exception as e:
        print(f"Error al convertir {nombre}: {e}")



Archivo business convertido y guardado como /content/sample_data/business.csv.
Archivo checkin convertido y guardado como /content/sample_data/checkin.csv.
Archivo review convertido y guardado como /content/sample_data/review.csv.
Archivo tip convertido y guardado como /content/sample_data/tip.csv.
Archivo user convertido y guardado como /content/sample_data/user.csv.


Al archivo bussines  lo dividimos segun su categoria separando los datos del estado de Florida

In [ ]:
# Cargamos la ruta del archivo business con los datos de Florida
ruta_business = "/content/sample_data/yelp_academic_dataset_business.json"

# Definimos las categorías
categorias = {
    "restaurantes": [
        "restaurant", "cafe", "espresso bar", "dart bar", "hookah bar", "cafeteria",
        "cocktail bar", "fast food", "pizza", "grill", "barbecue", "bistro", "brunch",
        "buffet", "bar"
    ],
    "educacion": [
        "school", "college", "private educational institution", "university",
        "kindergarten", "vocational school", "dance school", "music school",
        "education", "education center", "music conservatory", "music store",
        "music instructor"
    ],
    "supermercados_tiendas_negocios": [
        "supermarket", "grocery store", "food store", "bakery", "candy store",
        "ice cream shop", "convenience store", "boutique", "furniture store"
    ],
    "entretenimiento": [
        "cinema", "theater", "sports complex", "aquarium", "zoo", "museum",
        "concert", "gaming", "amusement park"
    ],
    "salud": [
        "hospital", "clinic", "dentist", "medical center", "pharmacy",
        "treatment center", "psychologist", "cardiologist", "nutritionist"
    ],
    "transporte": [
        "bus station", "train station", "airport", "taxi", "vehicle rental",
        "transportation service", "bicycle shop", "car rental"
    ],
    "servicios": [
        "government office", "bank", "atm", "post office", "insurance",
        "real estate", "law firm", "courier", "cleaning service"
    ],
    "servicios_animales": [
        "veterinary", "pet care", "animal hospital", "dog park", "pet boarding",
        "dog trainer", "animal shelter"
    ],
    "hoteles_y_hospedaje": [
        "hotel", "motel", "hostel", "bed & breakfast", "lodge", "resort",
        "vacation rental"
    ],
    "otros": []
}

business_df = pd.read_json(ruta_business, lines=True)

# Filtramos solo los negocios que coresponden al estado de Florida
florida_business_df = business_df[business_df['state'] == 'FL']

# Creamos un directorio de salida para guardar los archivos .csv
output_dir = "/content/categorias_csv/"
os.makedirs(output_dir, exist_ok=True)

# Clasificamos los negocios de Florida en categorías y guardarlos como .csv
for categoria, keywords in categorias.items():
    if keywords:
        filtered_df = florida_business_df[florida_business_df['categories'].str.contains('|'.join(keywords), case=False, na=False)]
    else:
        # Filtramos los negocios que no encajan en ninguna categoría
        todas_las_categorias = '|'.join(sum(categorias.values(), []))
        filtered_df = florida_business_df[~florida_business_df['categories'].str.contains(todas_las_categorias, case=False, na=False)]

    # Guardamos los datos filtrados en un archivo .csv
    output_path = os.path.join(output_dir, f"{categoria}.csv")
    filtered_df.to_csv(output_path, index=False)
    print(f"Categoría '{categoria}' guardada en: {output_path}")



Categoría 'restaurantes' guardada en: /content/categorias_csv/restaurantes.csv
Categoría 'educacion' guardada en: /content/categorias_csv/educacion.csv
Categoría 'supermercados_tiendas_negocios' guardada en: /content/categorias_csv/supermercados_tiendas_negocios.csv
Categoría 'entretenimiento' guardada en: /content/categorias_csv/entretenimiento.csv
Categoría 'salud' guardada en: /content/categorias_csv/salud.csv
Categoría 'transporte' guardada en: /content/categorias_csv/transporte.csv
Categoría 'servicios' guardada en: /content/categorias_csv/servicios.csv
Categoría 'servicios_animales' guardada en: /content/categorias_csv/servicios_animales.csv
Categoría 'hoteles_y_hospedaje' guardada en: /content/categorias_csv/hoteles_y_hospedaje.csv
Categoría 'otros' guardada en: /content/categorias_csv/otros.csv


•	Realizamos un merge con los datos de los archivo clasificado por negocio y el archivo checkin basándonos en la columna bussines_id

In [ ]:
# cargamos los directorios donde se encuentran  los achivos
input_dir = "/content/categorias_csv"
checkin_file = "/content/sample_data/checkin.csv"
output_dir = "/content/categorias1"
os.makedirs(output_dir, exist_ok=True)

# Cargamos el archivo checkin completo
print("Cargando datos de checkin...")
checkin_df = pd.read_csv(checkin_file)

# Procesamos cada archivo de categoría
for categoria_file in os.listdir(input_dir):
    if categoria_file.endswith(".csv"):
        try:
            categoria_path = os.path.join(input_dir, categoria_file)
            print(f"Procesando categoría: {categoria_file}...")

            categoria_df = pd.read_csv(categoria_path)

            # Realizamos el merge con el archivo checkin en business_id
            merged_df = pd.merge(categoria_df, checkin_df, on="business_id", how="inner")

            # Guardamos el resultado
            output_path = os.path.join(output_dir, categoria_file)
            merged_df.to_csv(output_path, index=False)
            print(f"Datos combinados guardados en: {output_path}")

        except Exception as e:
            print(f"Error procesando {categoria_file}: {e}")


Cargando datos de checkin...
Procesando categoría: salud.csv...
Datos combinados guardados en: /content/categorias1/salud.csv
Procesando categoría: otros.csv...
Datos combinados guardados en: /content/categorias1/otros.csv
Procesando categoría: hoteles_y_hospedaje.csv...
Datos combinados guardados en: /content/categorias1/hoteles_y_hospedaje.csv
Procesando categoría: restaurantes.csv...
Datos combinados guardados en: /content/categorias1/restaurantes.csv
Procesando categoría: entretenimiento.csv...
Datos combinados guardados en: /content/categorias1/entretenimiento.csv
Procesando categoría: supermercados_tiendas_negocios.csv...
Datos combinados guardados en: /content/categorias1/supermercados_tiendas_negocios.csv
Procesando categoría: servicios_animales.csv...
Datos combinados guardados en: /content/categorias1/servicios_animales.csv
Procesando categoría: educacion.csv...
Datos combinados guardados en: /content/categorias1/educacion.csv
Procesando categoría: transporte.csv...
Datos com

•	Realizamos un merge de los archivos del directorio categoría con el archivo review basados en la columna bussines_idy aplicamos un análisis de sentimientos ala columna text

In [ ]:
# cargamos los directorios de los archivos
input_dir = "/content/categorias1"
review_file = "/content/sample_data/review.csv"
output_dir = "/content/categorias2"
os.makedirs(output_dir, exist_ok=True)

# Función para mediante analisis de sentimientos calificar la satisfaccion del cliente
def clasificar_sentimiento(texto):
    polaridad = TextBlob(texto).sentiment.polarity
    if polaridad > 0:
        return 'positivo'
    elif polaridad < 0:
        return 'negativo'
    else:
        return 'neutral'

# Procesamos cada archivo de categoría
for categoria_file in os.listdir(input_dir):
    try:
        categoria_path = os.path.join(input_dir, categoria_file)
        print(f"Procesando categoría: {categoria_file}...")

        # Cargamos los datos por categoría
        categoria_df = pd.read_csv(categoria_path)

        # Leemos el archivo de reviews en chunks para evitar la saturar de la  memoria
        chunk_size = 100000
        processed_chunks = []
        for chunk in pd.read_csv(review_file, chunksize=chunk_size):
            # Mantenemos solo columnas necesarias
            chunk = chunk[['business_id', 'text']]

            # Realizalisamos elel merge
            merged_chunk = pd.merge(categoria_df, chunk, on="business_id", how="inner")

            # Clasificacion de sentimientos en cada chunk
            merged_chunk['sentiment'] = merged_chunk['text'].apply(clasificar_sentimiento)
            processed_chunks.append(merged_chunk)

        # Concatenanamos  todos los resultados de los chunks
        final_df = pd.concat(processed_chunks, ignore_index=True)

        # Guardamos resultados
        output_path = os.path.join(output_dir, f"{categoria_file.split('.')[0]}.csv")
        final_df.to_csv(output_path, index=False)
        print(f"Datos procesados y guardados en: {output_path}")

    except Exception as e:
        print(f"Error procesando {categoria_file}: {e}")



Procesando categoría: salud.csv...
Datos procesados y guardados en: /content/categorias2/salud.csv
Procesando categoría: otros.csv...
Datos procesados y guardados en: /content/categorias2/otros.csv
Procesando categoría: hoteles_y_hospedaje.csv...
Datos procesados y guardados en: /content/categorias2/hoteles_y_hospedaje.csv
Procesando categoría: restaurantes.csv...
Datos procesados y guardados en: /content/categorias2/restaurantes.csv
Procesando categoría: entretenimiento.csv...
Datos procesados y guardados en: /content/categorias2/entretenimiento.csv
Procesando categoría: supermercados_tiendas_negocios.csv...
Datos procesados y guardados en: /content/categorias2/supermercados_tiendas_negocios.csv
Procesando categoría: servicios_animales.csv...
Datos procesados y guardados en: /content/categorias2/servicios_animales.csv
Procesando categoría: educacion.csv...
Datos procesados y guardados en: /content/categorias2/educacion.csv
Procesando categoría: transporte.csv...
Datos procesados y gua

eliminamos filas repetidas repetidas

In [ ]:
# Cargamos los directorios
input_dir = '/content/categorias2'
output_dir = '/content/categorias2.5'

os.makedirs(output_dir, exist_ok=True)

# Procesamos cada archivo en el directorio
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        df = pd.read_csv(input_path)

        # Eliminamos filas duplicadas
        df = df.drop_duplicates()

        # Rellenamos datos vacíos con 'Null'
        df = df.fillna('Null')

        # Guardamos el archivo procesado
        df.to_csv(output_path, index=False)

        print(f"Archivo procesado y guardado: {output_path}")

print("Todos los archivos han sido procesados y guardados en 'categorias2.5'.")



Archivo procesado y guardado: /content/categorias2.5/salud.csv
Archivo procesado y guardado: /content/categorias2.5/otros.csv
Archivo procesado y guardado: /content/categorias2.5/hoteles_y_hospedaje.csv
Archivo procesado y guardado: /content/categorias2.5/restaurantes.csv
Archivo procesado y guardado: /content/categorias2.5/entretenimiento.csv
Archivo procesado y guardado: /content/categorias2.5/supermercados_tiendas_negocios.csv
Archivo procesado y guardado: /content/categorias2.5/servicios_animales.csv
Archivo procesado y guardado: /content/categorias2.5/educacion.csv
Archivo procesado y guardado: /content/categorias2.5/transporte.csv
Archivo procesado y guardado: /content/categorias2.5/servicios.csv
Todos los archivos han sido procesados y guardados en 'categorias2.5'.


In [ ]:
# Aumentamos el límite de tamaño de campo para evitar el error "field larger
csv.field_size_limit(sys.maxsize)

# Directorios de entrada y salida
input_dir = "/content/categorias2.5"
output_dir = "/content/categorias3"
os.makedirs(output_dir, exist_ok=True)

# establesemos el tamaño del chunk para evitar saturar la memoria
chunk_size = 10000

# Función para transformar y limpiar los datos por chunks
def transformar_datos_por_chunks(input_file, output_file):
    try:
        with pd.read_csv(input_file, chunksize=chunk_size, engine='python', on_bad_lines='skip') as reader:
            for i, chunk in enumerate(reader):
                chunk.columns = [col.strip().lower() for col in chunk.columns]
                if 'business_id' not in chunk.columns:
                    chunk['business_id'] = None

                # Cambiamos el nombre de las columnas a español para una mejor comprensión
                chunk['nombre'] = chunk['name']
                chunk['direccion'] = chunk['address']
                chunk['ciudad'] = chunk['city']
                chunk['estado'] = 'Florida'
                chunk['codigo_postal'] = chunk['postal_code']
                chunk['latitud'] = chunk['latitude']
                chunk['longitud'] = chunk['longitude']
                chunk['numero_de_reviews'] = chunk['review_count']
                chunk['atributos'] = chunk['attributes']
                chunk['categorias'] = chunk['categories']
                chunk['puntuacion_usuarios'] = chunk['stars']
                chunk['analisis_sentimientos'] = chunk['sentiment']
                chunk['url_del_negocio'] = None
                chunk['enlaces_google_maps'] = None
                chunk['anio'] = chunk.get('date', 'Desconocido')  # Para evitar errores si falta 'date'

                # Ordenamos las columnas
                columnas_finales = [
                    'business_id', 'nombre', 'direccion', 'ciudad', 'estado',
                    'codigo_postal', 'latitud', 'longitud',
                    'numero_de_reviews', 'atributos', 'categorias',
                    'puntuacion_usuarios', 'analisis_sentimientos',
                    'url_del_negocio', 'enlaces_google_maps', 'anio'
                ]
                chunk_final = chunk[columnas_finales]

                # Guardamos cada chunk en el archivo de salida
                mode = 'w' if i == 0 else 'a'
                chunk_final.to_csv(output_file, index=False, mode=mode, header=(i == 0))
                print(f"Chunk {i+1} procesado y guardado para {output_file}.")
    except KeyError as e:
        print(f"Error en el archivo {input_file}: {e}")
    except Exception as e:
        print(f"Error inesperado en el archivo {input_file}: {e}")

# Procesamos todos los archivos en la carpeta input_dir
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, file_name)
        print(f"Procesando {file_name}...")
        transformar_datos_por_chunks(input_file, output_file)

print("Todos los archivos han sido procesados exitosamente.")


Procesando salud.csv...
Chunk 1 procesado y guardado para /content/categorias3/salud.csv.
Chunk 2 procesado y guardado para /content/categorias3/salud.csv.
Procesando otros.csv...
Chunk 1 procesado y guardado para /content/categorias3/otros.csv.
Chunk 2 procesado y guardado para /content/categorias3/otros.csv.
Chunk 3 procesado y guardado para /content/categorias3/otros.csv.
Chunk 4 procesado y guardado para /content/categorias3/otros.csv.
Chunk 5 procesado y guardado para /content/categorias3/otros.csv.
Chunk 6 procesado y guardado para /content/categorias3/otros.csv.
Chunk 7 procesado y guardado para /content/categorias3/otros.csv.
Chunk 8 procesado y guardado para /content/categorias3/otros.csv.
Chunk 9 procesado y guardado para /content/categorias3/otros.csv.
Chunk 10 procesado y guardado para /content/categorias3/otros.csv.
Chunk 11 procesado y guardado para /content/categorias3/otros.csv.
Chunk 12 procesado y guardado para /content/categorias3/otros.csv.
Chunk 13 procesado y guar

Extraemos los años en que fueron tomados los datos

In [ ]:
# Directorios de entrada y de salida
input_dir = "/content/categorias3"
output_dir = "/content/categorias3.1"
os.makedirs(output_dir, exist_ok=True)

# Función para procesar un archivo
def procesar_archivo(input_file, output_file):
    df = pd.read_csv(input_file, low_memory=False)
    filas_expandidas = []
    for _, row in df.iterrows():
        if pd.notna(row['anio']):
            años = set(fecha.split('-')[0] for fecha in row['anio'].split(','))
            for año in años:
                nueva_fila = row.copy()
                nueva_fila['anio'] = año
                filas_expandidas.append(nueva_fila)
        else:
            filas_expandidas.append(row)

    # Crearmos un nuevo DataFrame con las filas expandidas
    df_expandidas = pd.DataFrame(filas_expandidas)

    # Eliminamos duplicados
    df_sin_duplicados = df_expandidas.drop_duplicates()

    # Guardamos el archivo procesado
    df_sin_duplicados.to_csv(output_file, index=False)
    print(f"Archivo procesado y guardado: {output_file}")

# Procesamos todos los archivos en el directorio
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, file_name)
        print(f"Procesando {file_name}...")
        procesar_archivo(input_file, output_file)

print("Todos los archivos han sido procesados exitosamente.")


Procesando salud.csv...
Archivo procesado y guardado: /content/categorias3.1/salud.csv
Procesando otros.csv...
Archivo procesado y guardado: /content/categorias3.1/otros.csv
Procesando hoteles_y_hospedaje.csv...
Archivo procesado y guardado: /content/categorias3.1/hoteles_y_hospedaje.csv
Procesando restaurantes.csv...
Archivo procesado y guardado: /content/categorias3.1/restaurantes.csv
Procesando entretenimiento.csv...
Archivo procesado y guardado: /content/categorias3.1/entretenimiento.csv
Procesando supermercados_tiendas_negocios.csv...
Archivo procesado y guardado: /content/categorias3.1/supermercados_tiendas_negocios.csv
Procesando servicios_animales.csv...
Archivo procesado y guardado: /content/categorias3.1/servicios_animales.csv
Procesando educacion.csv...
Archivo procesado y guardado: /content/categorias3.1/educacion.csv
Procesando transporte.csv...
Archivo procesado y guardado: /content/categorias3.1/transporte.csv
Procesando servicios.csv...
Archivo procesado y guardado: /co

Mediante el archivo de dimenciones ciudades_dim otorgamos los datos corespondientes a ciudad y estado

In [ ]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))
    return nombre.strip().lower()

# Función para corregir nombres específicos en la columna 'ciudad'
def corregir_ciudad(ciudad):
    if ciudad.lower() == "st petersburg":
        return "St. Petersburg"
    elif ciudad.lower() == "tampa":
        return "Tampa"
    elif ciudad.lower() == "land o lakes":
        return "Land O' Lakes"
    return ciudad

# Rutas de los directorios de entrada y salida
input_dir = "/content/categorias3.1"
output_dir = "/content/categorias3.2"
os.makedirs(output_dir, exist_ok=True)
unidades_dim_file = "/content/sample_data/ciudades_dim.csv"
unidades_dim = pd.read_csv(unidades_dim_file)

# Normalizamos y las columnas ciudad en anvos archivos
unidades_dim['ciudad'] = unidades_dim['ciudad'].apply(corregir_ciudad)
unidades_dim['ciudad_normalizada'] = unidades_dim['ciudad'].apply(normalizar_nombre)

# Procesamos cada archivo en el directorio de entrada
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, file_name)
        print(f"Procesando {file_name}...")
        df = pd.read_csv(input_file, low_memory=False)
        if 'ciudad' not in df.columns:
            print(f"Error: La columna 'ciudad' no existe en {file_name}.")
            continue
        df['ciudad'] = df['ciudad'].apply(corregir_ciudad)
        df['ciudad_normalizada'] = df['ciudad'].apply(normalizar_nombre)
        df = pd.merge(
            df,
            unidades_dim,
            how='left',
            on='ciudad_normalizada',
            suffixes=('', '_dim')
        )
        for col in ['id_condado', 'condado', 'codigo_postal_condado', 'latitud_condado',
                    'longitud_condado', 'id_ciudad', 'codigo_postal_ciudad', 'latitud_ciudad',
                    'longitud_ciudad']:
            if col not in df.columns:
                df[col] = None

        # Ordenamos las columnas
        columnas_ordenadas = [
            'id_nombre', 'nombre', 'direccion', 'id_condado', 'condado',
            'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
            'id_ciudad', 'ciudad', 'codigo_postal_ciudad', 'latitud_ciudad',
            'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
            'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
            'enlaces_google_maps', 'anio'
        ]

        for col in columnas_ordenadas:
            if col not in df.columns:
                df[col] = None

        df = df[columnas_ordenadas]

        # Guardamos el archivo procesado
        df.to_csv(output_file, index=False)
        print(f"Archivo procesado y guardado en: {output_file}")




Procesando salud.csv...
Archivo procesado y guardado en: /content/categorias3.2/salud.csv
Procesando otros.csv...
Archivo procesado y guardado en: /content/categorias3.2/otros.csv
Procesando hoteles_y_hospedaje.csv...
Archivo procesado y guardado en: /content/categorias3.2/hoteles_y_hospedaje.csv
Procesando restaurantes.csv...
Archivo procesado y guardado en: /content/categorias3.2/restaurantes.csv
Procesando entretenimiento.csv...
Archivo procesado y guardado en: /content/categorias3.2/entretenimiento.csv
Procesando supermercados_tiendas_negocios.csv...
Archivo procesado y guardado en: /content/categorias3.2/supermercados_tiendas_negocios.csv
Procesando servicios_animales.csv...
Archivo procesado y guardado en: /content/categorias3.2/servicios_animales.csv
Procesando educacion.csv...
Archivo procesado y guardado en: /content/categorias3.2/educacion.csv
Procesando transporte.csv...
Archivo procesado y guardado en: /content/categorias3.2/transporte.csv
Procesando servicios.csv...
Archiv

Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [43]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))  # Eliminar caracteres no alfanuméricos
    return nombre.strip().lower()  # Convertir a minúsculas y eliminar espacios

# Generamos ids alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

# Cargamos las rutas de los archivos de entrada y salida
input_file = "/content/categorias3.2/educacion.csv"
output_file = "/content/categorias3.3/educacion_ordenado.csv"
unidades_dim_file = "/content/drive/MyDrive/data/datos_prosesados_limpios/unidades_educativas_dim.csv"
centros_educacion_dim_file = "/content/categorias3.3/centros_educacion_dim.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)

# Renombramos columna 'nombre' a 'centros_educativos'
df.rename(columns={'nombre': 'centros_educativos'}, inplace=True)

# Normalizamos los nombres
df['centros_educativos_normalizado'] = df['centros_educativos'].apply(normalizar_nombre)

# Creamos un DataFrame con nombres únicos de centros educativos
centros_unicos = df[['centros_educativos_normalizado']].drop_duplicates().reset_index(drop=True)

# Generamos un id único para cada centro educativo
centros_unicos['id_centros_educativos'] = centros_unicos['centros_educativos_normalizado'].apply(lambda x: generar_id())

# Creamos el archivo centros_educacion_dim
centros_educacion_dim = centros_unicos[['id_centros_educativos', 'centros_educativos_normalizado']].rename(
    columns={'centros_educativos_normalizado': 'centros_educativos'}
)
centros_educacion_dim.to_csv(centros_educacion_dim_file, index=False)
print(f"Archivo centros_educacion_dim guardado en: {centros_educacion_dim_file}")

# Asignamos ids a los DataFrames originales
id_centros_dict = dict(zip(centros_educacion_dim['centros_educativos'], centros_educacion_dim['id_centros_educativos']))
df['id_centros_educativos'] = df['centros_educativos_normalizado'].map(id_centros_dict)

# Revisamos  y eliminamos duplicados considerando  las columnas'centros_educativos' y 'anio'
df = df.drop_duplicates(subset=['centros_educativos_normalizado', 'anio']).reset_index(drop=True)

# Ordenamos las columnas
columnas_ordenadas = [
    'id_centros_educativos', 'centros_educativos', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad','codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]
for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None

df = df[columnas_ordenadas]

# Agrupamos por 'anio'
df = df.sort_values(by=['anio', 'centros_educativos'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")





Archivo centros_educacion_dim guardado en: /content/categorias3.3/centros_educacion_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/educacion_ordenado.csv


Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [44]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))
    return nombre.strip().lower()

# Generamos un ids alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

# Cargamos los archivos de entrada y de salida
input_file = "/content/categorias3.2/entretenimiento.csv"
output_file = "/content/categorias3.3/entretenimiento_ordenado.csv"
centros_educacion_dim_file = "/content/categorias3.3/centros_entretenimiento_dim.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)
df.rename(columns={'nombre': 'centros_entretenimiento'}, inplace=True)

# Normalizar nombres
df['centros_entretenimiento_normalizado'] = df['centros_entretenimiento'].apply(normalizar_nombre)
centros_unicos = df[['centros_entretenimiento_normalizado']].drop_duplicates().reset_index(drop=True)
centros_unicos['id_centros_entretenimiento'] = centros_unicos['centros_entretenimiento_normalizado'].apply(lambda x: generar_id())
centros_entretenimiento_dim = centros_unicos[['id_centros_entretenimiento', 'centros_entretenimiento_normalizado']].rename(
    columns={'centros_entretenimiento_normalizado': 'centros_entretenimiento'}
)
centros_entretenimiento_dim.to_csv(centros_educacion_dim_file, index=False)
print(f"Archivo centros_entretenimiento_dim guardado en: {centros_educacion_dim_file}")

# Asignamos las ids a los DataFrames originales
id_centros_dict = dict(zip(centros_entretenimiento_dim['centros_entretenimiento'], centros_entretenimiento_dim['id_centros_entretenimiento']))
df['id_centros_entretenimiento'] = df['centros_entretenimiento_normalizado'].map(id_centros_dict)

# Revisamos y eliminamos duplicados considerando  las columnas'centros_entretenimiento' y 'anio'
if 'anio' in df.columns:
    df = df.drop_duplicates(subset=['centros_entretenimiento_normalizado', 'anio']).reset_index(drop=True)
else:
    print("Advertencia: La columna 'anio' no existe en el archivo de entrada.")

# Ordenamos las columnas
columnas_ordenadas = [
    'id_centros_entretenimiento', 'centros_entretenimiento', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad', 'latitud_ciudad','codigo_postal_ciudad', 'longitud_ciudad','estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]
for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None

df = df[columnas_ordenadas]
df = df.sort_values(by=['anio', 'centros_entretenimiento'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")


Archivo centros_entretenimiento_dim guardado en: /content/categorias3.3/centros_entretenimiento_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/entretenimiento_ordenado.csv


Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [45]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))
    return nombre.strip().lower()

# Generamos ids alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

#Cargamos los archivos de entrada y salida
input_file = "/content/categorias3.2/hoteles_y_hospedaje.csv"
output_file = "/content/categorias3.3/hosteleria_ordenado.csv"
centros_hosteleria_dim_file = "/content/categorias3.3/centros_hosteleria_dim.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)
df.rename(columns={'nombre': 'centros_hosteleria'}, inplace=True)

# Normalizamos los nombres
df['centros_hosteleria_normalizado'] = df['centros_hosteleria'].apply(normalizar_nombre)
centros_unicos = df[['centros_hosteleria_normalizado']].drop_duplicates().reset_index(drop=True)
centros_unicos['id_centros_hosteleria'] = centros_unicos['centros_hosteleria_normalizado'].apply(lambda x: generar_id())
centros_hosteleria_dim = centros_unicos[['id_centros_hosteleria', 'centros_hosteleria_normalizado']].rename(
    columns={'centros_hosteleria_normalizado': 'centros_hosteleria'}
)
centros_hosteleria_dim.to_csv(centros_hosteleria_dim_file, index=False)
print(f"Archivo centros_hosteleria_dim guardado en: {centros_hosteleria_dim_file}")
id_centros_dict = dict(zip(centros_hosteleria_dim['centros_hosteleria'], centros_hosteleria_dim['id_centros_hosteleria']))
df['id_centros_hosteleria'] = df['centros_hosteleria_normalizado'].map(id_centros_dict)
if 'anio' in df.columns:
    df = df.drop_duplicates(subset=['centros_hosteleria_normalizado', 'anio']).reset_index(drop=True)
else:
    print("Advertencia: La columna 'anio' no existe en el archivo de entrada.")
    df = df.drop_duplicates(subset=['centros_hosteleria_normalizado']).reset_index(drop=True)

#Ordenamos las columnas
columnas_ordenadas = [
    'id_centros_hosteleria', 'centros_hosteleria', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad','codigo_postal_ciudad',  'latitud_ciudad', 'longitud_ciudad','estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]

for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None
df = df[columnas_ordenadas]
if 'anio' in df.columns:
    df = df.sort_values(by=['anio', 'centros_hosteleria'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")


Archivo centros_hosteleria_dim guardado en: /content/categorias3.3/centros_hosteleria_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/hosteleria_ordenado.csv


Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [46]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))
    return nombre.strip().lower()

# Generamos ids  alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

#Cargamos los archivos de entrada y salida
input_file = "/content/categorias3.2/restaurantes.csv"
output_file = "/content/categorias3.3/restaurantes_ordenado.csv"
centros_restaurantes_dim_file = "/content/categorias3.3/centros_restaurantes_dim.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)
df.rename(columns={'nombre': 'centros_restaurantes'}, inplace=True)

# Normalizamos los  nombres
df['centros_restaurantes_normalizado'] = df['centros_restaurantes'].apply(normalizar_nombre)
centros_unicos = df[['centros_restaurantes_normalizado']].drop_duplicates().reset_index(drop=True)
centros_unicos['id_centros_restaurantes'] = centros_unicos['centros_restaurantes_normalizado'].apply(lambda x: generar_id())
centros_restaurantes_dim = centros_unicos[['id_centros_restaurantes', 'centros_restaurantes_normalizado']].rename(
    columns={'centros_restaurantes_normalizado': 'centros_restaurantes'}
)
centros_restaurantes_dim.to_csv(centros_restaurantes_dim_file, index=False)
print(f"Archivo centros_restaurantes_dim guardado en: {centros_restaurantes_dim_file}")
id_centros_dict = dict(zip(centros_restaurantes_dim['centros_restaurantes'], centros_restaurantes_dim['id_centros_restaurantes']))
df['id_centros_restaurantes'] = df['centros_restaurantes_normalizado'].map(id_centros_dict)

if 'anio' in df.columns:
    df = df.drop_duplicates(subset=['centros_restaurantes_normalizado', 'anio']).reset_index(drop=True)
else:
    print("Advertencia: La columna 'anio' no existe en el archivo de entrada.")
    df = df.drop_duplicates(subset=['centros_restaurantes_normalizado']).reset_index(drop=True)

# Ordenamos las columnas
columnas_ordenadas = [
    'id_centros_restaurantes', 'centros_restaurantes', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad', 'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad','estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]

for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None

df = df[columnas_ordenadas]
if 'anio' in df.columns:
    df = df.sort_values(by=['anio', 'centros_restaurantes'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")


Archivo centros_restaurantes_dim guardado en: /content/categorias3.3/centros_restaurantes_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/restaurantes_ordenado.csv


Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [48]:
#Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))  # Eliminar caracteres no alfanuméricos
    return nombre.strip().lower()  # Convertir a minúsculas y eliminar espacios

# Generamos ids alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

# Cargamos las rutas de los archivos de entrada y de salida
input_file = "/content/categorias3.2/supermercados_tiendas_negocios.csv"
output_file = "/content/categorias3.3/servicio_tiendas_negocio_ordenado.csv"
centros_tiendas_negocio_dim_file = "/content/categorias3.3/servicio_tiendas_negocio_dim.csv"

os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)
df.rename(columns={'nombre': 'centros_tiendas_negocio'}, inplace=True)
df['centros_tiendas_negocio_normalizado'] = df['centros_tiendas_negocio'].apply(normalizar_nombre)

centros_unicos = df[['centros_tiendas_negocio_normalizado']].drop_duplicates().reset_index(drop=True)
centros_unicos['id_centros_tiendas_negocio'] = centros_unicos['centros_tiendas_negocio_normalizado'].apply(lambda x: generar_id())
centros_tiendas_negocio_dim = centros_unicos[['id_centros_tiendas_negocio', 'centros_tiendas_negocio_normalizado']].rename(
    columns={'centros_tiendas_negocio_normalizado': 'centros_tiendas_negocio'}
)
centros_tiendas_negocio_dim.to_csv(centros_tiendas_negocio_dim_file, index=False)
print(f"Archivo servicio_tiendas_negocio_dim guardado en: {centros_tiendas_negocio_dim_file}")

id_centros_dict = dict(zip(centros_tiendas_negocio_dim['centros_tiendas_negocio'], centros_tiendas_negocio_dim['id_centros_tiendas_negocio']))
df['id_centros_tiendas_negocio'] = df['centros_tiendas_negocio_normalizado'].map(id_centros_dict)

# Revisamos y eliminamos duplicados considerando  las columnas'centros_tiendas_negocio' y 'anio'
if 'anio' in df.columns:
    df = df.drop_duplicates(subset=['centros_tiendas_negocio_normalizado', 'anio']).reset_index(drop=True)
else:
    print("Advertencia: La columna 'anio' no existe en el archivo de entrada.")
    df = df.drop_duplicates(subset=['centros_tiendas_negocio_normalizado']).reset_index(drop=True)

# Ordenamos las columnas
columnas_ordenadas = [
    'id_centros_tiendas_negocio', 'centros_tiendas_negocio', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad', 'codigo_postal_ciudad','latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]

for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None

df = df[columnas_ordenadas]

if 'anio' in df.columns:
    df = df.sort_values(by=['anio', 'centros_tiendas_negocio'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")



Archivo servicio_tiendas_negocio_dim guardado en: /content/categorias3.3/servicio_tiendas_negocio_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/servicio_tiendas_negocio_ordenado.csv


Otorgamos las ids corecpondiente mediante su archivo de dimenciones

In [49]:
# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    nombre = re.sub(r"[^a-zA-Z0-9\s]", "", str(nombre))  # Eliminar caracteres no alfanuméricos
    return nombre.strip().lower()  # Convertir a minúsculas y eliminar espacios

# Generamos ids alfanuméricos únicos de 5 dígitos
def generar_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

# cargamos las rutas de los archivos
input_file = "/content/categorias3.2/transporte.csv"
output_file = "/content/categorias3.3/servicio_transporte_ordenado.csv"
centros_transporte_dim_file = "/content/categorias3.3/servicio_transporte_dim.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df = pd.read_csv(input_file)

df.rename(columns={'nombre': 'transporte'}, inplace=True)

# Normalizamos los nombres
df['centros_transporte_normalizado'] = df['transporte'].apply(normalizar_nombre)

centros_unicos = df[['centros_transporte_normalizado']].drop_duplicates().reset_index(drop=True)
centros_unicos['id_transporte'] = centros_unicos['centros_transporte_normalizado'].apply(lambda x: generar_id())

# Creamos el archivo servicio_transporte_dim sin nombres repetidos
centros_transporte_dim = centros_unicos[['id_transporte', 'centros_transporte_normalizado']].rename(
    columns={'centros_transporte_normalizado': 'transporte'}
)
centros_transporte_dim.to_csv(centros_transporte_dim_file, index=False)
print(f"Archivo servicio_transporte_dim guardado en: {centros_transporte_dim_file}")
id_centros_dict = dict(zip(centros_transporte_dim['transporte'], centros_transporte_dim['id_transporte']))
df['id_transporte'] = df['centros_transporte_normalizado'].map(id_centros_dict)
if 'anio' in df.columns:
    df = df.drop_duplicates(subset=['centros_transporte_normalizado', 'anio']).reset_index(drop=True)
else:
    print("Advertencia: La columna 'anio' no existe en el archivo de entrada.")
    df = df.drop_duplicates(subset=['centros_transporte_normalizado']).reset_index(drop=True)

# Organisamos las columnas
columnas_ordenadas = [
    'id_transporte', 'transporte', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
    'id_ciudad', 'ciudad', 'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad','estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio',
    'enlaces_google_maps', 'anio'
]
for col in columnas_ordenadas:
    if col not in df.columns:
        df[col] = None
df = df[columnas_ordenadas]
if 'anio' in df.columns:
    df = df.sort_values(by=['anio', 'transporte'])

# Guardamos el archivo procesado agrupado por 'anio'
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")


Archivo servicio_transporte_dim guardado en: /content/categorias3.3/servicio_transporte_dim.csv
Archivo procesado y guardado en: /content/categorias3.3/servicio_transporte_ordenado.csv


In [50]:
import shutil
import os

# Ruta de origen y destino
origen = "/content/categorias3.3"
destino = "/content/drive/MyDrive/categorias3.3"

# Verificar si el destino existe y eliminarlo si es necesario
if os.path.exists(destino):
    shutil.rmtree(destino)  # Eliminar el directorio existente

# Mover el directorio
shutil.move(origen, destino)

print(f"Directorio movido de {origen} a {destino}")



Directorio movido de /content/categorias3.3 a /content/drive/MyDrive/categorias3.3
